In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [11]:
article_list = []

for file in os.listdir('/content/vietnews/data/train_tokenized'):
  #print(file)
  with open('/content/vietnews/data/train_tokenized/' + file, 'r') as f:
    article = f.read().splitlines()
    article_list.append(article[4:-1])   # leave out titles, abstracts, image subtitles
    f.close()

In [4]:
article_list[2]

['Lãnh_đạo Triều_Tiên_Kim_Jong-un ( trái ) và Tổng_thống Mỹ Donald_Trump trò_chuyện riêng tại hội_nghị thượng_đỉnh lần hai ở Hà_Nội hôm 28/2 .',
 '" Chúng_ta đã vượt qua đống tro tàn hậu_chiến và Cuộc Hành_quân Khổ_ải nhưng những thách_thức mà chúng_ta đang phải đối_mặt trong 10 năm trở_lại đây thực_tế là những khó_khăn nghiêm_trọng nhất trong lịch_sử ( nước Triều_Tiên ) cộng_hoà " , hãng tin Hàn_Quốc_Yonhap dẫn báo Rodong_Sinmun của đảng cầm_quyền Lao_động Triều_Tiên ngày 21/3 đưa tin .',
 '" Cuộc Hành_quân Khổ_ải " ám_chỉ giai_đoạn những năm 1990 của thế_kỷ trước khi Triều_Tiên vật_lộn với nạn đói và nhiều người đã vượt_biên để thoát khỏi cảnh đói_khát .',
 '" Chúng_ta đã chứng_minh rằng đến cuối_cùng chúng_ta có_thể tự mở ra con_đường rạng_rỡ phía trước dù đối_mặt với bất_cứ thách_thức nào " , tờ_báo này viết .',
 '" Tự_lực là một việc khó_khăn nhưng nó giống như một liều thuốc giúp tăng_cường sức_mạnh quốc_gia " .',
 'Tờ_báo này cho_rằng việc phụ_thuộc vào sự giúp_đỡ từ bên ngoài g

In [12]:
for i in range(len(article_list)):
  article_list[i] = list(filter(None, article_list[i]))
  article_list[i] = ' '.join(article_list[i])
  #article_list[i] = [x.replace('_', ' ') for x in article_list[i]]
  article_list[i] = article_list[i].replace('_', ' ')

In [13]:
article_list[2]

'Lãnh đạo Triều Tiên Kim Jong-un ( trái ) và Tổng thống Mỹ Donald Trump trò chuyện riêng tại hội nghị thượng đỉnh lần hai ở Hà Nội hôm 28/2 . " Chúng ta đã vượt qua đống tro tàn hậu chiến và Cuộc Hành quân Khổ ải nhưng những thách thức mà chúng ta đang phải đối mặt trong 10 năm trở lại đây thực tế là những khó khăn nghiêm trọng nhất trong lịch sử ( nước Triều Tiên ) cộng hoà " , hãng tin Hàn Quốc Yonhap dẫn báo Rodong Sinmun của đảng cầm quyền Lao động Triều Tiên ngày 21/3 đưa tin . " Cuộc Hành quân Khổ ải " ám chỉ giai đoạn những năm 1990 của thế kỷ trước khi Triều Tiên vật lộn với nạn đói và nhiều người đã vượt biên để thoát khỏi cảnh đói khát . " Chúng ta đã chứng minh rằng đến cuối cùng chúng ta có thể tự mở ra con đường rạng rỡ phía trước dù đối mặt với bất cứ thách thức nào " , tờ báo này viết . " Tự lực là một việc khó khăn nhưng nó giống như một liều thuốc giúp tăng cường sức mạnh quốc gia " . Tờ báo này cho rằng việc phụ thuộc vào sự giúp đỡ từ bên ngoài giống như " thuốc độc 

In [18]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(article_list)

In [20]:
print(tfidf_matrix.shape)

(105418, 97249)


Kết thúc preprocessing

In [16]:
# index_ = 0

# for i_article in article_list[:5]:

#   sentence_list = i_article.split('.')
#   sentence_score_list = []

#   for i_sentence in sentence_list:
#     i_sentence_score = 0
#     for i_word.lower() in i_sentence.split(' '):
#       i_sentence_score += tfidf_matrix[index_, tfidf.vocabulary_[i_word]]
#     sentence_score_list.append(i_sentence_score)
#   index += 1



['Cảnh sát Canada vừa công bố nguyên nhân cái chết của vợ chồng tỉ phú Barry Sherman và bà Honey , song kết luận sơ bộ của vụ việc chỉ góp thêm bí ẩn cho vụ việc chấn động này ', ' Nhà sáng lập hãng dược Apotex , ông Barry Sherman , 75 tuổi , và vợ , bà Honey Sherman , 70 tuổi , được tìm thấy đã chết tại nhà riêng ở North York hôm 15-12 ', ' Cảnh sát Toronto đánh giá vụ việc là " đáng ngờ " nên đã yêu cầu lực lượng điều tra án mạng vào cuộc , theo thông tin từ Đài CBC News ( Canada ) ngày 18-12 ', ' Trong thông cáo phát đi hôm 17-12 ( giờ địa phương ) , cảnh sát Toronto tuyên bố cả hai nạn nhân đều bị " ngạt thở do dây buộc " , nghĩa là bị xiết cổ , cho đến chết ', ' Tuy nhiên , điều khó hiểu là các điều tra viên tuyên bố họ sẽ không tìm kiếm thêm nghi can nào khác trong vụ việc , theo Sky News ', ' Một nguồn tin cảnh sát trước đó cho CBC News biết một người ngoài gia đình Sherman đã phát hiện ra thi thể vợ chồng tỉ phú , song cảnh sát không tiết lộ gì nhiều về quá trình điều tra ', ' 

In [30]:
tfidf_matrix[1,tfidf.vocabulary_['nhật']]

0.2055191032930217

In [ ]:
def score_sentence(sentence, a_index_):
  sentence_score = 0
  for i_word.lower() in i_sentence.split(' '):
      sentence_score += tfidf_matrix[a_index_, tfidf.vocabulary_[i_word]]
  
  return sentence_score


def score_article(article):
  sentence_score_list = []
  sentence_list = article.split('.')

  for i_sentence in sentence_list:
    sentence_score = score_sentence(i_sentence)
    sentence_score_list.append(sentence_score)

  return sentence_score_list, sentence_list


def get_summary(sentence_score_list, sentence_list):
  mean_score = np.mean(sentence_score_list)
  sentence_score_list > mean_score


In [36]:
mean = 11
score_list = [1,2, 3,4, 578, 12, 324]


TypeError: ignored